**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.**

In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install inspyred

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import inspyred
from random import Random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# Genetic Algorithms and Optimization

We have already considered some optimization methods in earlier notebooks, including **ordinary least squares** (for linear regression) and **gradient descent** (for logistic regression, neural nets, etc.). We can use ordinary least squares to compute a closed-form solution to linear regression. For non-linear systems we usually can't give a closed-form solution and the optimization needs to be done iteratively: and gradient descent is one of the methods that can be applied.

We have also mentioned one of the disadvantages of gradient-based optimization method: for non-convex functions, they are not guaranteed to find the global optimum. They can get trapped in a local optimum, or even a saddle point. A further disadvantage that we have not discussed as much is that the approach is not easy to apply when it is hard to get a good analytic description of the objective function and/or the objective function is not differentiable.

In the next few notebooks we will consider a different kind of optimization methods: the so-called evolutionary methods, especially genetic algorithms. These methods do not guarantee that the global optimum will be found either, but they are less likely to get trapped in local optima and they can often get reasonable results in not too long a time. They do not have any very strong requirements concerning the structure of the problem, they work with non-differentiable functions, etc. For this reason, they might be worth looking into for some problems.

On the other hand, evolutionary methods are typically not especially effective. When other approaches to the same problem exist, they will often be much more effective than an evolutionary approach. Sometimes one can achieve good results by hybridizing some such existing solution with an evolutionary approach.

These notebooks will show how to apply some simple evolutionary approaches in Python.

## The Objective Function

As usual, we will start by defining the objective function that we are going to be optimizing.

In [ ]:
def criterion(x, y):
    nom = 1 - (np.sin(0.5 * np.sqrt(x**2 + y**2)))**2
    denom =  1 + 0.001*(x**2 + y**2)
    return 1 - nom / denom

Let us visualize the function to make things more concrete:

In [ ]:
fig = plt.figure(figsize=(8, 6))
xx, yy = np.mgrid[-20:20.05:0.25, -20:20.05:0.25]
zz = criterion(xx, yy)

ax = plt.subplot(111, projection='3d')
ax.plot_surface(xx, yy, zz, rstride=1, cstride=1, cmap='Spectral')

plt.xlabel('$x$')
plt.ylabel('$y$')
ax.set_zlabel("$f(x, y)$")

As we can see, the function has a number of minima. However, there is just one global minimum, which is at (0, 0).

## Genetic Algorithms

Next we will define a few components that genetic algorithms require.

### Generating the Individuals

Let's start with the function that generates individuals:

In [ ]:
def generator(random, args):
    return [random.uniform(-20, 20), random.uniform(-20, 20)]

### The Fitness Function

Next the function which evaluates the population. Let's iterate over all the individuals and call function ``criterion`` for each of them:

In [ ]:
def evaluator(candidates, args):
    return [criterion(*can) for can in candidates]

### Running the Genetic Algorithm

Next we are ready to run the genetic algorithm itself.

In [ ]:
ga = inspyred.ec.GA(Random())
# We will want to archive the best solution
ga.archiver = inspyred.ec.archivers.best_archiver
# We will use blend_crossover and gaussian_mutation as our genetic operators
ga.variator = [inspyred.ec.variators.blend_crossover, 
               inspyred.ec.variators.gaussian_mutation]
# We will use tournament selection as our selection method
ga.selector = inspyred.ec.selectors.tournament_selection
# Evolution will terminate after a certain number of generations
ga.terminator = inspyred.ec.terminators.generation_termination

# We run the evolution
final_pop = ga.evolve(generator=generator, # generating the initial population
                      evaluator=evaluator, # evaluating populations
                      pop_size=100, # population size
                      maximize=False, # are we maximizing or minimizing?
                      max_generations=200, # the maximum number of generations
                      num_elites=1, # the number of elite individuals (they will get
                                    # copied into the next generation without modification)
                      tournament_size=3 # size of the tournament (for tournament selection)
            )

## Evaluating the Results

We will next print the fittest individual from the archive and its fitness:

In [ ]:
best_ind = ga.archive[0]

print("Ind.: {}\nFitness: {}".format(
    best_ind.candidate, best_ind.fitness
))

We can also plot all the individuals from the last generation (in red) and the fittest individual (in green).

In [ ]:
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, zz, 20, cmap='Blues')
plt.colorbar(label="z")

points = np.array([ii.candidate for ii in final_pop])

plt.scatter(
    points[:, 0], points[:, 1],
    linewidths=1, edgecolors='k',
    s=80,
    color='r'
)

plt.scatter(
    best_ind.candidate[0],
    best_ind.candidate[1],
    linewidths=1, edgecolors='k',
    s=100,
    color='g'
)

plt.xlabel("x"); plt.ylabel("y")